In [1]:
# Import required modules
import pandas as pd

In [2]:
# Read in main data
main_df = pd.read_csv("_Bucket 3.1 - Sheet1.csv")

# Create an indicator column
main_df["ind"] = range(1, main_df.shape[0]+1)

# Split out shutterstock data
shutter_only = main_df[main_df["Image Name"].str.contains("shutterstock", na=False, case=False)]
# Create page link for shutterstock to make requests to
shutter_only["created_link"] = "https://www.shutterstock.com/image-photo/" + shutter_only["Image Name"].str.split("_").str[-1].str.replace(".jpg", "")

# Split out adobestock data
adobe_only = main_df[main_df["Image Name"].str.contains("adobestock", na=False, case=False)]
# Create page link for adobestock to make requests to
adobe_only["created_link"] = "https://stock.adobe.com/" + adobe_only["Image Name"].str.split("_").str[-1].str.replace(".jpeg", "")


# Split out records except shutter and adobe
not_shutter_adobe = main_df[~main_df["Image Name"].str.contains("shutterstock|adobestock", case=False, na=False)]


# Merge stutter, adobe and not shutter-adobe after creating links
main_with_link = pd.concat([shutter_only, adobe_only, not_shutter_adobe]).sort_values("ind").reset_index(drop=True)

<ipython-input-2-fc37d06e0f92>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  shutter_only["created_link"] = "https://www.shutterstock.com/image-photo/" + shutter_only["Image Name"].str.split("_").str[-1].str.replace(".jpg", "")
<ipython-input-2-fc37d06e0f92>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shutter_only["created_link"] = "https://www.shutterstock.com/image-photo/" + shutter_only["Image Name"].str.split("_").str[-1].str.replace(".jpg", "")
<ipython-input-2-fc37d06e0f92>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  adobe_only["created_link"] = "https://stock.adobe.com/" + adobe_only["Image Name"].str.split("_").str[-1].str.r

In [3]:
# Read in shutter and adobe stock scraped data
shutter_scraped = pd.read_csv("bucket_3.1_shutterstock_scraped.csv")
adobe_scraped = pd.read_csv("bucket_3.1_adobestock_scraped.csv")

# Merge scraped shutter and adobe
merged_scraped = pd.concat([shutter_scraped, adobe_scraped])
merged_scraped.keywords = merged_scraped.keywords.str.replace("[", "").str.replace("'", "").str.replace("]", "")

<ipython-input-3-10401a69cd8c>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  merged_scraped.keywords = merged_scraped.keywords.str.replace("[", "").str.replace("'", "").str.replace("]", "")


In [4]:
# Seperate main data with and without created link
main_with_created_link = main_with_link[main_with_link.created_link.notna()]
main_without_created_link = main_with_link[main_with_link.created_link.isna()]

In [5]:
# Merge on created link
merged_on_created_link = pd.merge(main_with_created_link, merged_scraped, on="created_link", how="left")


# Final merge
final_merge = pd.concat([merged_on_created_link, main_without_created_link]).sort_values("ind")

# Keep only the required columns
final_merge = final_merge.loc[:, ["Course Name", "Module Name", "Image Name", 
                                  "Date", "Size", "URL", "Description",
                                 "ind", "created_link", "title", "category", "keywords", "source_link"]]

In [6]:
# Rename columns
final_merge.columns = [
    "Course Name", 
    "Module Name",
    "Image Name",
    "Date",
    "Size",
    "URL",
    "Description",
    "ind",
    "Shutterstock Link",
    "Image Title",
    "Category",
    "Keywords",
    "Shutterstock Preview Image Link"
]

# Reorganize columns as intended orders
final_merge = final_merge.loc[:, ["Course Name", "Module Name", "Image Name", "Date", "Size",
                                  "URL", "Description", "Image Title", "Category", "Keywords",
                                  "Shutterstock Link", "Shutterstock Preview Image Link"]]

In [7]:
final_merge.to_excel("bucket_3.1_alligned.xlsx", index=None)